# Artificial Neural Network

### Importing the libraries

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [12]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [13]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values #x dataset; starting from 3rd column until the end BUT NOT the last one with the EXITED
y = dataset.iloc[:, -1].values #y variable; the last column 
'''we don't need all columns, e.g. the customer ID or their surnames'''

"we don't need all columns, e.g. the customer ID or their surnames"

In [14]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [15]:
print(y) #the decision of the customer to leave the bank or not 

[1 0 1 ... 1 1 0]


### Encoding categorical data

Converting categorical text data into model-understandable numerical data

Label Encoding the "Gender" column

So we need to encode the gender column possibly into 0 and 1 

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #the gender column :, 2 meaning from the beginning till the 2nd column (the Gender one)

In [17]:
print(X) # female and male with 0 and 1, a random decision of the machine 

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

We have three countries; France, Spain and Germany - we cannot use 0,1 and 2, the way we did with Gender. Why use OneHotEncoder: depending on the data we have, we might run into situations where, after label encoding, we might confuse our model into thinking that a column has data with some kind of order or hierarchy, when we clearly don’t have it. To avoid this, we ‘OneHotEncode’ that column. What is does is: it takes a column which has categorical data, which has been label encoded, and then splits the column into multiple columns. The numbers are replaced by 1s and 0s, depending on which column has what value. In our example, we’ll get three new columns, one for each country — France, Germany, and Spain.

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

'changing the index of the column we want to transform'

'changing the index of the column we want to transform'

In [19]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


How the encoded column looks like now; the 1.0 0.0 1.0 is for France, 1.0 0.0 1.0 is for Spain and 0.0 1.0 0.0 is for Germany. 

### Splitting the dataset into the Training set and Test set

Note: Setting random_state a fixed value will guarantee that same sequence of random numbers are generated each time you run the code. And unless there is some other randomness present in the process, the results produced will be same as always. This helps in verifying the output.

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

### *It is compulsory for deep learning*

What is it? Feature scaling is the process of normalising the range of features in a dataset, in order for the ML model to interpret the features on the same scale. StandardScaler is the Scikit-learn function for standardisation.


In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

Initializes our ANN as a sequence of layers

In [22]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [24]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# units; the number of neurons we want to have in this hidden layer

### Adding the second hidden layer

In [25]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [27]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# using the sigmoid one because we want a binary outcome
# it gives the probability of whether each customer leaves the bank or not 

## Part 3 - Training the ANN

### Compiling the ANN

Compiling it with an optimizer, loss and metrics --> 
We then compile the model with the compile function. This creates the neural network model by specifying the details of the learning process. The model hasn’t been trained yet. Right now we’re just declaring the optimizer to use and the loss function to minimize. The arguments for the compile function are defined as follows:
*`optimizer:`* Which optimizer to use in order to minimize the loss function. There are a lot of different optimizers, most of them based on gradient descent. We will explore different optimizers in another tutorial. For now we will use the adam optimizer, which is the one people prefer to use by default. The 'adam' one is prefered because it can perform stochastic gradient descent, meaning that it will update the weights in order to reduce the loss error between our predictions and the real results.
*`loss:`* The loss function to minimize. Since we’re building a binary 0/1 classifier, the loss function to minimize is binary_crossentropy. It is the way to compute the difference between the predictions and the real result. For non binary classification we would have used crossentropy.
*(NOTE: When predicting more than two categories the activation function should not be sigmoid but softmax.) *
*`metrics:`* Which metric to report statistics on, for classification problems we set this as accuracy.

In [28]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

The method to train whatever ML model, is always the same, the **fit** method, which also always takes the same parameters. The first one is the x_train--> the matrix of features of the training set, then y_train--> for the dependent variable of the training set.

Also two more parameters: the **batch size** which gives exactly the number of predictions we want to have in the batch, compared to that same number of real results. The classic value of the batch size is 32. The batch size is a hyperparameter of gradient descent that controls the number of training samples to work through before the model’s internal parameters are updated.

number of **epochs**: An epoch means training the neural network with all the training data for one cycle. In an epoch, we use all of the data exactly once. A forward pass and a backward pass together are counted as one pass: An epoch is made up of one or more batches, where we use a part of the dataset to train the neural network. The number of epochs is a hyperparameter of gradient descent that controls the number of complete passes through the training dataset.

In [29]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 4s 3ms/step - loss: 0.5166 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4770 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4469 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4338 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4273 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4222 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4174 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4103 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4017 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3930 - accura

Our accuracy around 86% means that out of 100 observations, we have 86 correct predictions. 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [30]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

**Important note 3:** why > 0.5? we choose a threshold of  0.5 to say that if that predicted probability is larger than 0.5, we will consider the final result to be 1, meaning that there is more than 50 percent chance that the predicted outcome is one. And however, if the predicted probability that the customer leaves the bank is below 0.5, we will consider it to be zero. Of course, you can choose a different value of the threshold, especially when you have critical outcome for a critical decision and in that case, you can, for example, increase the threshold.

**Important note 4:** remember to apply scaling from your sc object because your artificial neural network was trained with scaled values.

### Predicting the Test set results

Back to our model now. 

In [31]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

checking our results

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1523   72]
 [ 210  195]]


0.859

We have 1523 correct predictions that the customer stays in the bank, 72 incorect predictions that the customer leaves the bank, 210 correct predictions that the customer will leave the bank and 195 incorrect predictions that the customer stays in the bank. 
Accuracy of a 85,9% - almost 86% is a very decent accuracy 